# Write a python program to display IMDB’s Top rated 100 Indian movies’ data
https://www.imdb.com/list/ls056092300/ (i.e. name, rating, year ofrelease) and make data frame.

In [5]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd


# Downloading imdb top 100 movie's data
url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
movies = soup.select('td.titleColumn')
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value')
		for b in soup.select('td.posterColumn span[name=ir]')]




# create a empty list for storing
# movie information
list = []

# Iterating over movies to extract
# each movie's details
for index in range(0, len(movies)):
	
	# Separating movie into: 'place',
	# 'title', 'year'
	movie_string = movies[index].get_text()
	movie = (' '.join(movie_string.split()).replace('.', ''))
	movie_title = movie[len(str(index))+1:-7]
	year = re.search('\((.*?)\)', movie_string).group(1)
	place = movie[:len(str(index))-(len(movie))]
	data = {"place": place,
			"movie_title": movie_title,
			"rating": ratings[index],
			"year": year,
			"star_cast": crew[index],
			}
	list.append(data)

# printing movie details with its rating.
for movie in list:
	print(movie['place'], '-', movie['movie_title'], '('+movie['year'] +
		') -', 'Starring:', movie['star_cast'], movie['rating'])


##.......##
df = pd.DataFrame(list)
df.to_csv('imdb_top_100_movies.csv',index=False)


# Write a python program to scrape details of all the posts from https://www.patreon.com/coreyms .Scrape the
heading, date, content and the likes for the video from the link for the youtube video from the post.

In [6]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.patreon.com/coreyms'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

posts = soup.find_all('div', class_='post')

for post in posts:
  heading = post.find('h3').text
  date = post.find('time')['datetime']
  content = post.find('div', class_='post__content').text
  youtube_link = post.find('a', class_='post__youtube-link')['href']
  likes = post.find('span', class_='post__likes').text

  print(f'Heading: {heading}')
  print(f'Date: {date}')
  print(f'Content: {content}')
  print(f'YouTube Link: {youtube_link}')
  print(f'Likes: {likes}')
  print('\n')

# Write a python program to scrape house details from mentioned URL. It should include house title, location,
area, EMI and price from https://www.nobroker.in/ .Enter three localities which are Indira Nagar, Jayanagar,
Rajaji Nagar.

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to get house details from a given URL
def get_house_details(url):
    response = requests.get(url)
    response.raise_for_status()  # Check for request errors
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Lists to hold house details
    titles = []
    locations = []
    areas = []
    emis = []
    prices = []
    
    # Find all house listings on the page
    listings = soup.find_all('div', class_='nb__1pO3e')

    for listing in listings:
        # Extract house title
        title_tag = listing.find('h2', class_='nb__1wYw4')
        title = title_tag.text.strip() if title_tag else 'N/A'
        
        # Extract house location
        location_tag = listing.find('div', class_='nb__1CSfO')
        location = location_tag.text.strip() if location_tag else 'N/A'
        
        # Extract area
        area_tag = listing.find('div', class_='nb__2eG9F')
        area = area_tag.text.strip() if area_tag else 'N/A'
        
        # Extract EMI (if available)
        emi_tag = listing.find('div', class_='nb__2_OuS')
        emi = emi_tag.text.strip() if emi_tag else 'N/A'
        
        # Extract price
        price_tag = listing.find('div', class_='nb__2i5Zt')
        price = price_tag.text.strip() if price_tag else 'N/A'
        
        # Append details to lists
        titles.append(title)
        locations.append(location)
        areas.append(area)
        emis.append(emi)
        prices.append(price)
    
    # Create a DataFrame
    data = {
        'Title': titles,
        'Location': locations,
        'Area': areas,
        'EMI': emis,
        'Price': prices
    }
    df = pd.DataFrame(data)
    
    return df

# Localities to search for
localities = ['Indira Nagar', 'Jayanagar', 'Rajaji Nagar']
base_url = 'https://www.nobroker.in/'

# DataFrame to collect data from all localities
all_data = pd.DataFrame()

# Fetch details for each locality
for locality in localities:
    search_url = f"{base_url}/search?query={locality.replace(' ', '%20')}"
    print(f"Fetching data for: {locality}")
    locality_df = get_house_details(search_url)
    locality_df['Locality'] = locality
    all_data = pd.concat([all_data, locality_df], ignore_index=True)

# Display the DataFrame
print(all_data)

# Optionally, save to a CSV file
all_data.to_csv('house_details_nobroker.csv', index=False)


Fetching data for: Indira Nagar
Fetching data for: Jayanagar
Fetching data for: Rajaji Nagar
Empty DataFrame
Columns: [Title, Location, Area, EMI, Price, Locality]
Index: []


In [10]:
import requests
from bs4 import BeautifulSoup

# URL of the Bewakoof Bestsellers page
url = 'https://www.bewakoof.com/bestseller?sort=popular'

# Function to scrape product details from the Bewakoof page
def get_product_details(url):
    # Send a GET request to the URL
    response = requests.get(url)
    response.raise_for_status()  # Check for request errors
    
    # Parse the page content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Lists to hold product details
    product_names = []
    prices = []
    image_urls = []
    
    # Find all product containers
    product_containers = soup.find_all('div', class_='product')
    
    # Scrape details for the first 10 products
    for container in product_containers[:10]:
        # Extract product name
        name_tag = container.find('h2', class_='product-name')
        product_name = name_tag.text.strip() if name_tag else 'N/A'
        
        # Extract price
        price_tag = container.find('span', class_='product-price')
        price = price_tag.text.strip() if price_tag else 'N/A'
        
        # Extract image URL
        image_tag = container.find('img', class_='product-image')
        image_url = image_tag['src'] if image_tag else 'N/A'
        
        # Append details to lists
        product_names.append(product_name)
        prices.append(price)
        image_urls.append(image_url)
    
    # Return the scraped data
    return {
        'Product Name': product_names,
        'Price': prices,
        'Image URL': image_urls
    }

# Get product details
product_details = get_product_details(url)

# Print product details
for i in range(len(product_details['Product Name'])):
    print(f"Product Name: {product_details['Product Name'][i]}")
    print(f"Price: {product_details['Price'][i]}")
    print(f"Image URL: {product_details['Image URL'][i]}")
    print('---')

# Optionally, you can save the data to a CSV file using pandas
import pandas as pd

df = pd.DataFrame(product_details)
df.to_csv('bewakoof_bestsellers.csv', index=False)


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the CNBC World news page
url = 'https://www.cnbc.com/world/?region=world'

# Function to get news details from the CNBC page
def get_news_details(url):
    # Send a GET request to the URL
    response = requests.get(url)
    response.raise_for_status()  # Check for request errors
    
    # Parse the page content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Lists to hold news details
    headings = []
    dates = []
    links = []
    
    # Find all news articles
    articles = soup.find_all('article', class_='LatestNews-article')
    
    # Extract details from each article
    for article in articles:
        # Extract the headline
        heading_tag = article.find('a', class_='LatestNews-headline')
        heading = heading_tag.text.strip() if heading_tag else 'N/A'
        
        # Extract the date (if available)
        date_tag = article.find('time')
        date = date_tag.text.strip() if date_tag else 'N/A'
        
        # Extract the news link
        link_tag = article.find('a', class_='LatestNews-headline')
        link = link_tag['href'] if link_tag and link_tag['href'].startswith('https') else 'N/A'
        
        # Append details to lists
        headings.append(heading)
        dates.append(date)
        links.append(link)
    
    # Return the scraped data
    return {
        'Heading': headings,
        'Date': dates,
        'Link': links
    }

# Get news details
news_details = get_news_details(url)

# Print news details
for i in range(len(news_details['Heading'])):
    print(f"Heading: {news_details['Heading'][i]}")
    print(f"Date: {news_details['Date'][i]}")
    print(f"Link: {news_details['Link'][i]}")
    print('---')

# Optionally, you can save the data to a CSV file using pandas
df = pd.DataFrame(news_details)
df.to_csv('cnbc_world_news.csv', index=False)


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the KeAi Publishing most downloaded articles page
url = 'https://www.keaipublishing.com/en/journals/artificial-intelligence-in-agriculture/most-downloaded-articles/'

# Function to get article details from the KeAi Publishing page
def get_article_details(url):
    # Send a GET request to the URL
    response = requests.get(url)
    response.raise_for_status()  # Check for request errors
    
    # Parse the page content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Lists to hold article details
    titles = []
    dates = []
    authors = []
    
    # Find all article containers
    article_containers = soup.find_all('div', class_='article-card')
    
    # Extract details from each article
    for container in article_containers:
        # Extract the title
        title_tag = container.find('a', class_='title')
        title = title_tag.text.strip() if title_tag else 'N/A'
        
        # Extract the publication date
        date_tag = container.find('div', class_='meta-info')
        date = date_tag.text.strip().split('|')[0].strip() if date_tag else 'N/A'
        
        # Extract the authors
        author_tags = container.find('div', class_='authors')
        authors_list = [author.text.strip() for author in author_tags.find_all('a')] if author_tags else []
        author_names = ', '.join(authors_list)
        
        # Append details to lists
        titles.append(title)
        dates.append(date)
        authors.append(author_names)
    
    # Return the scraped data
    return {
        'Title': titles,
        'Date': dates,
        'Authors': authors
    }

# Get article details
article_details = get_article_details(url)

# Print article details
for i in range(len(article_details['Title'])):
    print(f"Title: {article_details['Title'][i]}")
    print(f"Date: {article_details['Date'][i]}")
    print(f"Authors: {article_details['Authors'][i]}")
    print('---')

# Optionally, save the data to a CSV file using pandas
df = pd.DataFrame(article_details)
df.to_csv('keaai_publishing_articles.csv', index=False)
